In [1]:
import pandas as pd
import datetime
import calendar
import requests

def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)


#парсит с https://www.gismeteo.ru/diary/ID_ГОРОДА/ГОД/МЕСЯЦ/
#search_city - str, название города
#begin_date, end_date - datetime, даты начала парса и конца парса соответственно
#для работы требуется приложенный файл meteo_stations.csv
def parse_wheather(search_city,begin_date,end_date):  
    headers ={
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.116 Safari/537.36 OPR/40.0.2308.81',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'DNT': '1',
        'Accept-Encoding': 'gzip, deflate, lzma, sdch',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4'
    }#сервер хитрый, без хедера сервер режет соединение
    cities_df = pd.read_csv('meteo_stations.csv', delimiter=',')
    city_id = cities_df.loc[cities_df.name == search_city].id.item()
    iter_date = begin_date
    ret_df = pd.DataFrame()
    while(iter_date.strftime('%Y')*10000+iter_date.strftime('%m')*100 <= 10000*end_date.strftime('%Y')+100*end_date.strftime('%m')):
        current_y = int(iter_date.strftime('%Y'))
        current_m = int(iter_date.strftime('%m'))
        
        #парсим
        url = 'https://www.gismeteo.ru/diary/'+str(city_id)+'/'+str(current_y)+'/'+str(current_m)+'/'
        response = requests.get(url,headers=headers)
        got_df = pd.read_html(response.content)[0]
        got_df.columns = ['date','day_temp','day_pressure','NaN_day_clouds','NaN_day_rain','day_wind','evening_temp','evening_pressure','NaN_evening_clouds','NaN_evening_rain','evening_wind']
        
        #чистим хлам
        del got_df['NaN_day_clouds'],got_df['NaN_day_rain'], got_df['day_wind'],got_df['NaN_evening_clouds'],got_df['NaN_evening_rain'],got_df['evening_wind']
        del got_df['day_pressure'], got_df['evening_pressure'] #давление. может быть пригодится, хз
        got_df = got_df.drop(len(got_df)-1, axis=0)
        got_df = got_df.drop(len(got_df)-1, axis=0)
        
        #число месяца в нормальную дату
        got_df['date'] = pd.to_datetime((current_y*10000+current_m*100+got_df.date.astype(int)).apply(str),format='%Y%m%d')
        ret_df = ret_df.append(got_df, ignore_index=True, sort=False)
        iter_date = add_months(iter_date,1)
    #убираем лишние даты
    begin_date = pd.to_datetime(begin_date)
    end_date = pd.to_datetime(end_date)
    ret_df.drop(ret_df[ret_df.date < begin_date].index, inplace=True)
    ret_df.drop(ret_df[ret_df.date > end_date].index, inplace=True)
    ret_df = ret_df.reset_index(drop=True)
    return ret_df

In [3]:
#пример работы
first_date = datetime.date(2020, 2, 19)
last_date = datetime.date(2020,10,20)
df = parse_wheather('Архангельск', first_date,last_date)
df

,date,day_temp,evening_temp
0,2020-02-19,+2,+2
1,2020-02-20,-2,-10
2,2020-02-21,-1,0
3,2020-02-22,+3,+3
4,2020-02-23,+2,+2
...,...,...,...
240,2020-10-16,+2,+1
241,2020-10-17,+1,+1
242,2020-10-18,+2,+1
243,2020-10-19,0,-2
